In [1]:
from pyathena import connect
import pandas as pd
import os
import re 

#Assume role before!
conn = connect(aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
              aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
              aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
              s3_staging_dir='s3://aws-athena-query-results-878189897945-eu-west-1/',
              region_name='eu-west-1')

query = "SELECT * FROM alpha_health_evermind.events"
df = pd.read_sql(query, conn)
df

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,createdat,deploymentenv,device,event,product,schema,source,type,unique_name,user,...,view,data,element,error,level,message,stacktrace,year,month,day
0,2019-11-16T08:04:30.729961+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",app_open,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_open.null.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,None,None,None,None,None,None,None,2019,11,16
1,2019-11-16T08:04:31.119303+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",app_resume,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_resume.null.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,None,None,None,None,None,None,None,2019,11,16
2,2019-11-16T08:04:31.122276+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",screen_view,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,screen_view.splash.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,splash,None,None,None,None,None,None,2019,11,16
3,2019-11-16T08:04:31.124669+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",screen_view,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,screen_view.splash.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,splash,"{view=splash, question=null, answers=null, sco...",None,None,None,None,None,2019,11,16
4,2019-11-16T08:06:00.430315+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",tap,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,tap.splash.sign-up,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,splash,None,sign-up,None,None,None,None,2019,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87053,2019-11-26T12:00:09.483744+00:00,prod,"{id=2777CD3D-7AA8-4461-BB54-DC7338E51EC1, mode...",app_pause,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_pause.activity.null,{id=c7d94497-16a6-4983-bbd8-d49e5939c3b4},...,activity,None,None,None,None,None,None,2019,11,26
87054,2019-11-26T12:00:09.483744+00:00,prod,"{id=2777CD3D-7AA8-4461-BB54-DC7338E51EC1, mode...",app_pause,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_pause.activity.null,{id=c7d94497-16a6-4983-bbd8-d49e5939c3b4},...,activity,None,None,None,None,None,None,2019,11,26
87055,2019-11-26T14:43:13.389037+00:00,prod,"{id=a9a1e3f5498c6aac, model=SM-G950F, platform...",app_resume,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_resume.assessment.null,{id=6a252400-4aaf-46ab-aeb7-6788d4bb140f},...,assessment,None,None,None,None,None,None,2019,11,26
87056,2019-11-26T14:48:27.728179+00:00,prod,"{id=a9a1e3f5498c6aac, model=SM-G950F, platform...",app_pause,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_pause.assessment.null,{id=6a252400-4aaf-46ab-aeb7-6788d4bb140f},...,assessment,None,None,None,None,None,None,2019,11,26


In [2]:
toy_data = df.copy()
toy_data['createdat'] = pd.to_datetime(toy_data.createdat, utc = True)
toy_data['prev_mytimestamp'] = toy_data.groupby(['userid'])['createdat'].transform(lambda x:x.shift(1))

In [3]:
toy_data

,createdat,deploymentenv,device,event,product,schema,source,type,unique_name,user,...,data,element,error,level,message,stacktrace,year,month,day,prev_mytimestamp
0,2019-11-16 08:04:30.729961+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",app_open,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_open.null.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,None,None,None,None,None,None,2019,11,16,NaT
1,2019-11-16 08:04:31.119303+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",app_resume,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_resume.null.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,None,None,None,None,None,None,2019,11,16,2019-11-16 08:04:30.729961+00:00
2,2019-11-16 08:04:31.122276+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",screen_view,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,screen_view.splash.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,None,None,None,None,None,None,2019,11,16,2019-11-16 08:04:31.119303+00:00
3,2019-11-16 08:04:31.124669+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",screen_view,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,screen_view.splash.null,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,"{view=splash, question=null, answers=null, sco...",None,None,None,None,None,2019,11,16,2019-11-16 08:04:31.122276+00:00
4,2019-11-16 08:06:00.430315+00:00,prod,"{id=A01CFEC1-B0EA-4866-8739-940D6A507080, mode...",tap,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,tap.splash.sign-up,{id=A01CFEC1-B0EA-4866-8739-940D6A507080},...,None,sign-up,None,None,None,None,2019,11,16,2019-11-16 08:04:31.124669+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87053,2019-11-26 12:00:09.483744+00:00,prod,"{id=2777CD3D-7AA8-4461-BB54-DC7338E51EC1, mode...",app_pause,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_pause.activity.null,{id=c7d94497-16a6-4983-bbd8-d49e5939c3b4},...,None,None,None,None,None,None,2019,11,26,2019-11-26 10:10:04.094351+00:00
87054,2019-11-26 12:00:09.483744+00:00,prod,"{id=2777CD3D-7AA8-4461-BB54-DC7338E51EC1, mode...",app_pause,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_pause.activity.null,{id=c7d94497-16a6-4983-bbd8-d49e5939c3b4},...,None,None,None,None,None,None,2019,11,26,2019-11-26 12:00:09.483744+00:00
87055,2019-11-26 14:43:13.389037+00:00,prod,"{id=a9a1e3f5498c6aac, model=SM-G950F, platform...",app_resume,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_resume.assessment.null,{id=6a252400-4aaf-46ab-aeb7-6788d4bb140f},...,None,None,None,None,None,None,2019,11,26,2019-11-26 14:57:29.548980+00:00
87056,2019-11-26 14:48:27.728179+00:00,prod,"{id=a9a1e3f5498c6aac, model=SM-G950F, platform...",app_pause,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_pause.assessment.null,{id=6a252400-4aaf-46ab-aeb7-6788d4bb140f},...,None,None,None,None,None,None,2019,11,26,2019-11-26 14:43:13.389037+00:00


In [4]:
from datetime import timedelta

T = timedelta(seconds=30*60)

toy_data['new_session'] = ((toy_data['createdat']
                            - pd.to_datetime(toy_data.prev_mytimestamp))>=T).astype(int)


In [5]:
toy_data['increment'] = toy_data.groupby("userid")['new_session'].cumsum()
toy_data['session_id'] = toy_data['userid'].astype(str) + '_' + toy_data['increment'].astype(str)

In [6]:
toy_data = toy_data.sort_values(['createdat'])

In [7]:
toy_data

,createdat,deploymentenv,device,event,product,schema,source,type,unique_name,user,...,level,message,stacktrace,year,month,day,prev_mytimestamp,new_session,increment,session_id
7027,2019-11-12 14:32:29.344228+00:00,stage,"{id=e48cdd0765262bbc, model=Android SDK built ...",app_resume,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_resume.home.null,{id=758277e0-a73b-4e3a-9368-928d61ea407b},...,None,None,None,2019,11,18,NaT,0,0,758277e0-a73b-4e3a-9368-928d61ea407b_0
7028,2019-11-12 14:32:30.893367+00:00,stage,"{id=e48cdd0765262bbc, model=Android SDK built ...",app_pause,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_pause.home.null,{id=758277e0-a73b-4e3a-9368-928d61ea407b},...,None,None,None,2019,11,18,2019-11-12 14:32:29.344228+00:00,0,0,758277e0-a73b-4e3a-9368-928d61ea407b_0
243,2019-11-15 11:28:12.078122+00:00,prod,"{id=2251df37935a0656, model=Pixel 3a, platform...",app_open,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,app_open.null.null,{id=2251df37935a0656},...,None,None,None,2019,11,15,NaT,0,0,2251df37935a0656_0
244,2019-11-15 11:28:14.255037+00:00,prod,"{id=2251df37935a0656, model=Pixel 3a, platform...",screen_view,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,screen_view.splash.null,{id=2251df37935a0656},...,None,None,None,2019,11,15,2019-11-15 11:28:12.078122+00:00,0,0,2251df37935a0656_0
245,2019-11-15 11:28:14.383942+00:00,prod,"{id=2251df37935a0656, model=Pixel 3a, platform...",screen_view,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,screen_view.splash.null,{id=2251df37935a0656},...,None,None,None,2019,11,15,2019-11-15 11:28:14.255037+00:00,0,0,2251df37935a0656_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32231,2020-03-07 23:52:59.435611+00:00,prod,"{id=91475485-49AF-47B2-89B8-45B7774338BC, mode...",tap,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,tap.add_record_activity.write-feedback,{id=53801b9e-49b5-4b08-b964-4adfa1f965b2},...,None,None,None,2020,03,07,2020-03-07 23:52:57.900506+00:00,0,1,53801b9e-49b5-4b08-b964-4adfa1f965b2_1
32232,2020-03-07 23:54:08.626495+00:00,prod,"{id=91475485-49AF-47B2-89B8-45B7774338BC, mode...",tap,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,tap.add_record_activity.input-done,{id=53801b9e-49b5-4b08-b964-4adfa1f965b2},...,None,None,None,2020,03,07,2020-03-07 23:52:59.435611+00:00,0,1,53801b9e-49b5-4b08-b964-4adfa1f965b2_1
32233,2020-03-07 23:54:10.193443+00:00,prod,"{id=91475485-49AF-47B2-89B8-45B7774338BC, mode...",tap,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,tap.add_record_activity.done,{id=53801b9e-49b5-4b08-b964-4adfa1f965b2},...,None,None,None,2020,03,07,2020-03-07 23:54:08.626495+00:00,0,1,53801b9e-49b5-4b08-b964-4adfa1f965b2_1
32234,2020-03-07 23:54:10.203096+00:00,prod,"{id=91475485-49AF-47B2-89B8-45B7774338BC, mode...",screen_view,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,screen_view.activity_outro.null,{id=53801b9e-49b5-4b08-b964-4adfa1f965b2},...,None,None,None,2020,03,07,2020-03-07 23:54:10.193443+00:00,0,1,53801b9e-49b5-4b08-b964-4adfa1f965b2_1


In [8]:
toy_data[toy_data['userid'] == '41e1dcca-20c7-4027-b36b-4386740a535b']['session_id'].sort_values().unique()

array(['41e1dcca-20c7-4027-b36b-4386740a535b_0',
       '41e1dcca-20c7-4027-b36b-4386740a535b_1',
       '41e1dcca-20c7-4027-b36b-4386740a535b_10',
       '41e1dcca-20c7-4027-b36b-4386740a535b_11',
       '41e1dcca-20c7-4027-b36b-4386740a535b_12',
       '41e1dcca-20c7-4027-b36b-4386740a535b_13',
       '41e1dcca-20c7-4027-b36b-4386740a535b_14',
       '41e1dcca-20c7-4027-b36b-4386740a535b_15',
       '41e1dcca-20c7-4027-b36b-4386740a535b_16',
       '41e1dcca-20c7-4027-b36b-4386740a535b_17',
       '41e1dcca-20c7-4027-b36b-4386740a535b_18',
       '41e1dcca-20c7-4027-b36b-4386740a535b_19',
       '41e1dcca-20c7-4027-b36b-4386740a535b_2',
       '41e1dcca-20c7-4027-b36b-4386740a535b_20',
       '41e1dcca-20c7-4027-b36b-4386740a535b_21',
       '41e1dcca-20c7-4027-b36b-4386740a535b_22',
       '41e1dcca-20c7-4027-b36b-4386740a535b_23',
       '41e1dcca-20c7-4027-b36b-4386740a535b_24',
       '41e1dcca-20c7-4027-b36b-4386740a535b_25',
       '41e1dcca-20c7-4027-b36b-4386740a535b_26',
   

In [9]:
def getKey(k, raw):
    """
    Function used to return the content for different keys in the data array. 
    This function will be used to create new columns for scores etc.
    """
    if isinstance(raw, float):
        return(float('NaN'))
    
    raw.replace('null', 'NaN')
    
    if k == 'score':
        num = re.search('{}=(\d+)'.format(k), raw)
        if num:
          num = int(num.group(1))
        return(num)

    if k == 'answers': 
        ary = re.search(r"\[(.*)\]".format(k), raw)
        print(ary)
        if ary:
            ary = ary.group()
            ary = re.sub('\D', ' ', ary)
            ary = ary.split()
            if ary:
                return(ary)
    
    # if string
    sub_string = re.search('{}=(.+?),'.format(k), raw)
    if sub_string:
      string = sub_string.group(1)
      if sub_string:
        return(string)


In [10]:
df2 = toy_data.copy()
df2.fillna(value = pd.np.nan, inplace=True)
general = df2['data']
df2['score'] = general.apply(lambda x: getKey('score', x))

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [21]:
df2

,createdat,deploymentenv,device,event,product,schema,source,type,unique_name,user,...,stacktrace,year,month,day,prev_mytimestamp,new_session,increment,session_id,score,activity-title
167,2019-11-15 13:23:42.856116+00:00,prod,"{id=07B2B14E-7728-4DB8-8BEC-EC03B0B013DB, mode...",assessment,"{id=com.wingmanalpha.app.evermind, name=Runner...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=e2b25de5-444e-4f7c-82f2-a2207181a93a},...,NaN,2019,11,15,2019-11-15 13:23:41.628228+00:00,0,0,e2b25de5-444e-4f7c-82f2-a2207181a93a_0,21.0,NaN
5115,2019-11-18 09:03:30.347480+00:00,prod,"{id=e48cdd0765262bbc, model=Android SDK built ...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=9130c977-71cd-4b6e-a8e0-6d269e2e0022},...,NaN,2019,11,18,2019-11-18 09:03:28.970216+00:00,0,0,9130c977-71cd-4b6e-a8e0-6d269e2e0022_0,20.0,NaN
4772,2019-11-18 09:03:30.347480+00:00,prod,"{id=e48cdd0765262bbc, model=Android SDK built ...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=9130c977-71cd-4b6e-a8e0-6d269e2e0022},...,NaN,2019,11,18,2019-11-18 09:03:28.970216+00:00,0,0,9130c977-71cd-4b6e-a8e0-6d269e2e0022_0,20.0,NaN
53466,2019-11-20 11:00:57.050878+00:00,prod,"{id=6F380742-C788-42F7-9A56-090BF2B76E0F, mode...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=c137634b-03c7-439c-b504-6dbe2374c4e9},...,NaN,2019,11,20,2019-11-20 11:00:56.054452+00:00,0,3,c137634b-03c7-439c-b504-6dbe2374c4e9_3,17.0,NaN
60617,2019-11-20 15:19:22.430658+00:00,prod,"{id=9c47cf0b47d5c9c5, model=SM-G925F, platform...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=b9226e9d-f839-46e9-8e2b-e27fa42736aa},...,NaN,2019,11,20,2019-11-20 15:19:16.476046+00:00,0,33,b9226e9d-f839-46e9-8e2b-e27fa42736aa_33,30.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65956,2020-03-03 23:05:59.978795+00:00,prod,"{id=e76b9b512982887a, model=Redmi Note 8 Pro, ...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=41e1dcca-20c7-4027-b36b-4386740a535b},...,NaN,2020,03,03,2020-03-03 23:05:52.939504+00:00,0,61,41e1dcca-20c7-4027-b36b-4386740a535b_61,8.0,NaN
20600,2020-03-04 22:22:29.998268+00:00,prod,"{id=CC08C60B-4BCD-4347-9987-2D80B04525C9, mode...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=fb6b5135-d438-4b7c-b3aa-7f0bd286e0f0},...,NaN,2020,03,04,2020-03-04 22:22:25.850248+00:00,0,0,fb6b5135-d438-4b7c-b3aa-7f0bd286e0f0_0,9.0,NaN
50548,2020-03-05 17:24:10.107584+00:00,prod,"{id=EC3D969C-24F0-4307-A48C-D554FCCBA062, mode...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=fa9bcb7d-8a6c-4e4e-a6e3-1465a6f8fc1b},...,NaN,2020,03,05,2020-03-05 17:24:08.322423+00:00,0,0,fa9bcb7d-8a6c-4e4e-a6e3-1465a6f8fc1b_0,16.0,NaN
24094,2020-03-06 19:37:54.287608+00:00,prod,"{id=f2ef124ce52edcc3, model=Mi 9T Pro, platfor...",assessment,"{id=com.wingmanalpha.app.evermind, name=Evermi...",/schemas/events/analytics-device-event/0.json,analytics,track,assessment.assessment.null,{id=798790c2-c8b1-468d-b061-5ede38be36bc},...,NaN,2020,03,06,2020-03-06 19:37:42.459873+00:00,0,10,798790c2-c8b1-468d-b061-5ede38be36bc_10,14.0,NaN


In [ ]:
def getDuration (min, max):
    
    
    return duration

In [28]:
max = []
max = df2.groupby('session_id')['createdat'].max()

min = []
min = df2.groupby('session_id')['createdat'].min()

diff = max - min
diff

session_id
00da7219-c1bd-4b67-acc5-fa6da0668d05_1           0 days 00:00:00
00da7219-c1bd-4b67-acc5-fa6da0668d05_14   0 days 00:02:20.382681
00da7219-c1bd-4b67-acc5-fa6da0668d05_2           0 days 00:00:00
00da7219-c1bd-4b67-acc5-fa6da0668d05_6           0 days 00:00:00
00da7219-c1bd-4b67-acc5-fa6da0668d05_9    1 days 12:38:33.225114
                                                   ...          
fde78b86-e4f9-4546-853b-6882c8b4f085_3           0 days 00:00:00
fde78b86-e4f9-4546-853b-6882c8b4f085_31   0 days 00:04:40.035832
fde78b86-e4f9-4546-853b-6882c8b4f085_32   0 days 00:14:16.930351
fde78b86-e4f9-4546-853b-6882c8b4f085_4    0 days 00:00:03.000709
fde78b86-e4f9-4546-853b-6882c8b4f085_6           0 days 00:00:00
Name: createdat, Length: 407, dtype: timedelta64[ns]

In [11]:
df2['activity-title'] = general.apply(lambda x: getKey('activity-title', x))

In [12]:
df2 = df2.replace('null', pd.np.nan)

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


In [13]:
def outputPerUser(df, cat):
    df.dropna(subset = [cat], inplace=True)
    
    df.sort_values(["session_id", cat])
    valid_ids = df['session_id'].unique()
    new = df[(df['session_id'].isin(valid_ids))]
    new = new.groupby('session_id')
    new = new[cat, 'activity-title'].agg(list) 
    return new

In [14]:
scoresdf = outputPerUser(df2, 'score')

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [15]:
scoresdf.sort_values('score')

,score,activity-title
session_id,,
4e672aa1-7d57-48cb-8765-2e703c7944e6_1,[0.0],[Match the shapes]
2397e1e3-e7b4-459a-82c6-c8a69cdc7031_11,[0.0],[Match the shapes]
1843b45d-4580-4969-bf0a-318a3bce4b32_17,[0.0],[Match the shapes]
1843b45d-4580-4969-bf0a-318a3bce4b32_16,[0.0],[Match the shapes]
1843b45d-4580-4969-bf0a-318a3bce4b32_15,[0.0],[Match the shapes]
...,...,...
2397e1e3-e7b4-459a-82c6-c8a69cdc7031_71,[32.0],[nan]
fde78b86-e4f9-4546-853b-6882c8b4f085_14,"[32.0, 32.0]","[Match the shapes, Match the shapes]"
a7bf7453-a776-4e35-ac5d-a766a36b98d2_1,[34.0],[nan]


In [16]:
import statistics 

def meanScore(x):
    m = statistics.mean(x)
    return m

In [17]:
general = scoresdf['score']
scoresdf['meanScore'] = general.apply(lambda x: meanScore(x))

In [18]:
scoresdf

,score,activity-title,meanScore
session_id,,,
00da7219-c1bd-4b67-acc5-fa6da0668d05_1,[23.0],[nan],23.0
00da7219-c1bd-4b67-acc5-fa6da0668d05_14,"[0.0, 0.0, 0.0]","[Match the shapes, Match the shapes, Match the...",0.0
00da7219-c1bd-4b67-acc5-fa6da0668d05_2,[0.0],[Match the shapes],0.0
00da7219-c1bd-4b67-acc5-fa6da0668d05_6,[20.0],[nan],20.0
00da7219-c1bd-4b67-acc5-fa6da0668d05_9,"[0.0, 7.0, 7.0, 0.0, 0.0]","[Match the shapes, Match the shapes, Match the...",2.8
...,...,...,...
fde78b86-e4f9-4546-853b-6882c8b4f085_3,[0.0],[Match the shapes],0.0
fde78b86-e4f9-4546-853b-6882c8b4f085_31,"[0.0, 0.0, 0.0, 0.0, 15.0, 15.0]","[Match the shapes, Match the shapes, Match the...",5.0
fde78b86-e4f9-4546-853b-6882c8b4f085_32,"[0.0, 0.0, 0.0, 0.0, 0.0]","[Match the shapes, Match the shapes, Match the...",0.0


In [ ]:
if meanScore > 20:
    